In [1]:
!pip install transformers

In [2]:
!pip install evaluate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 527.3/527.3 kB 25.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 8.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 10.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 13.1 MB/s eta 0:00:00
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 14.0.2
    Uninstalling pyarrow-14.0.2:
      Successfully uninstalled pyarrow-14.0.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 24.4.1 requires pyarrow<15.0.0a0,>=14.0.1, but you have pyarrow 17.0.0 which is incompatible.
ibis-framework 8.0.0 requires pyarrow<16,>=2, but you have pyarrow 17.0.0 which is 

In [3]:
!pip install rouge

In [4]:
#!pip install --upgrade pyarrow


In [5]:
!pip install evaluate

In [1]:
# *QA System MultiChoice*

# **QA System MultiChoice**

##### Authors:
- Barba Farro, José Luis
- Risco Cosavalente, Mariana
- Quispe Carhuapoma, Sharon

## Librerías

In [6]:
import torch
import json
from tqdm import tqdm
import torch.nn as nn
from torch.optim import Adam
import nltk
import spacy
import string
import evaluate  # Bleu
from torch.utils.data import Dataset, DataLoader, RandomSampler
import pandas as pd
import numpy as np
import transformers
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from transformers import T5Tokenizer, T5Model, T5ForConditionalGeneration, T5TokenizerFast

import warnings
warnings.filterwarnings("ignore")

## **MQA  Dataset**

In [7]:
import re
import pandas as pd


file_path = "/data/train - train.txt"

regex = r'^(?P<text>.*?)\t(?P<question>.*?)\t(?P<A>.*?)\t(?P<B>.*?)\t(?P<C>.*?)\t(?P<D>.*?)\t(?P<E>.*?)\t(?P<answer>[A-E])\t(?P<reason>.*)$'

data_processed = []

with open(file_path, 'r', encoding='utf-8') as file:
    header = file.readline().strip().split('\t')
    for line in file:
        line = line.strip()
        match = re.match(regex, line)
        if match:
            row_dict = match.groupdict()
            data_processed.append(row_dict)
df = pd.DataFrame(data_processed)
csv_path = "/data/train_processed.csv"
df.to_csv(csv_path, index=False, encoding='utf-8')
print(f'DataFrame creado y guardado en {csv_path}')

DataFrame creado y guardado en /content/train_processed.csv


In [8]:
df["answer"].value_counts()

,count
answer,
C,1367
B,1264
A,1224
D,1219
E,1205


In [9]:
import re
import pandas as pd


file_path = "/data/dev - dev.txt"

regex = r'^(?P<text>.*?)\t(?P<question>.*?)\t(?P<A>.*?)\t(?P<B>.*?)\t(?P<C>.*?)\t(?P<D>.*?)\t(?P<E>.*?)\t(?P<answer>[A-E])\t(?P<reason>.*)$'

data_processed = []

with open(file_path, 'r', encoding='utf-8') as file:
    header = file.readline().strip().split('\t')
    for line in file:
        line = line.strip()
        match = re.match(regex, line)
        if match:
            row_dict = match.groupdict()
            data_processed.append(row_dict)
df_test = pd.DataFrame(data_processed)
csv_path = "/data/test_processed.csv"
df_test.to_csv(csv_path, index=False, encoding='utf-8')
print(f'DataFrame creado y guardado en {csv_path}')

DataFrame creado y guardado en /content/test_processed.csv


In [10]:
df_test.head()

,text,question,A,B,C,D,E,answer,reason
0,No sabemos si Dios existe. Por esto se plantea...,"Principalmente,el autor intenta dilucidar",el argumento de la prueba ontológica de la exi...,los distintos postulados teológicos a lo largo...,la necesidad de referirse a Dios en el sistema...,la esencia divina a la luz de los planteamient...,el círculo vicioso que se aprecia en las defin...,A,El texto nos introduce en el tema teológico pa...
1,No sabemos si Dios existe. Por esto se plantea...,Se infiere que la esencia de un objeto cualquiera,es refutada por todos los filósofos.,puede ser percibida por Dios.,genera gran polémica científica.,se manifiesta en su definición.,resulta fascinante e irritante.,D,"Tanto la esencia de Dios,de los triángulos y d..."
2,No sabemos si Dios existe. Por esto se plantea...,"En el texto,la palabra CUESTIÓN significa",oposición.,gresca.,asunto.,pregunta.,posición.,D,La cuestión planteada implica la pregunta de s...
3,No sabemos si Dios existe. Por esto se plantea...,Resulta compatible sostener que la definición ...,no amerita ningún tipo de planteamiento filosó...,incluye como característica su natural imperfe...,debe armonizar con los planteamientos de la ló...,implica que de la nada puede aparecer un mundo.,niega como característica primordial su eterni...,C,En todos los distintos planteamientos de la pr...
4,"""Las ballenas que están estrechamente relacion...",La idea principal del texto sostiene que,las ballenas asesinas orcas y los delfines mul...,las orcas son capaces de aprender nuevos sonid...,las ballenas que viven juntas producen llamada...,durante mucho tiempo se pensó que las ballenas...,las ballenas tienen un alto nivel de plasticid...,B,El texto explica como las ballenas fueron capa...


In [11]:
df_test["answer"].value_counts()

,count
answer,
C,166
B,150
A,143
D,135
E,130


In [12]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6279 entries, 0 to 6278
Data columns (total 9 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   text      6279 non-null   object
 1   question  6279 non-null   object
 2   A         6279 non-null   object
 3   B         6279 non-null   object
 4   C         6279 non-null   object
 5   D         6279 non-null   object
 6   E         6279 non-null   object
 7   answer    6279 non-null   object
 8   reason    6279 non-null   object
dtypes: object(9)
memory usage: 441.6+ KB


In [13]:
import re
import pandas as pd


file_path = "/data/test - test.txt"

regex = r'^(?P<text>.*?)\t(?P<question>.*?)\t(?P<A>.*?)\t(?P<B>.*?)\t(?P<C>.*?)\t(?P<D>.*?)\t(?P<E>.*?)\t(?P<answer>[A-E])\t(?P<reason>.*)$'

data_processed = []

with open(file_path, 'r', encoding='utf-8') as file:
    header = file.readline().strip().split('\t')
    for line in file:
        line = line.strip()
        match = re.match(regex, line)
        if match:
            row_dict = match.groupdict()
            data_processed.append(row_dict)
df_summit = pd.DataFrame(data_processed)
csv_path = "/data/summit_processed.csv"
df_summit.to_csv(csv_path, index=False, encoding='utf-8')
print(f'DataFrame creado y guardado en {csv_path}')

DataFrame creado y guardado en /content/summit_processed.csv


In [14]:
train_data = df
test_data = df_test
summit_data = df_summit

In [15]:
print(f"train_data shape {train_data.shape}")
print(f"test_data shape {test_data.shape}")
print(f"summit_data shape {summit_data.shape}")

train_data shape (6279, 9)
test_data shape (724, 9)
summit_data shape (720, 9)


In [16]:
summit_data.head()

,text,question,A,B,C,D,E,answer,reason
0,La identificación de autores y el estudio de l...,La finalidad del autor del texto es,señalar la importancia de la visión universali...,proponer que las literaturas regionales están ...,denunciar la visión centralista y hegemónica d...,señalar el gran auge del fenómeno regionalista...,aclarar el sentido y la importancia de los est...,E,La intención del autor es aclarar el sentido e...
1,La identificación de autores y el estudio de l...,"En el texto,el verbo ENTRONIZAR alude a",una posición privilegiada.,una muestra de solidez.,la adquisición de fortuna.,un signo de inclusión.,una fama inmerecida.,A,Se refiere a que estos discursos ocupan una po...
2,La identificación de autores y el estudio de l...,"En el texto,el término CUERPO alude a",sensualidad.,títulos.,novelas.,extrañeza.,conjunto.,E,"Se refiere al conjunto de obras literarias.,,,..."
3,La identificación de autores y el estudio de l...,Marque la alternativa que es incompatible con ...,Hay quienes consideran que las literaturas reg...,La literatura regional pretende fortalecer los...,La constitución del canon literario halla su f...,Las visiones periféricas del mundo no suelen s...,Mostrar las expresiones autóctonas fue uno de ...,B,Es el regionalismo el que tiene ese objetivo. ...
4,La identificación de autores y el estudio de l...,Si “literatura regional” fuese sinónimo de “re...,resaltarían las características vernáculas de ...,destacarían la influencia de los escritores ex...,destacarían la influencia de los escritores na...,criticarían las propuestas que fuesen ajenas a...,adoptarían una perspectiva latinoamericanista ...,A,"Si fuesen sinónimos,entonces la “literatura re..."


In [17]:
def show_one(example):
    print(f"Context: {example['text']}")
    print(f"Question: {example['question']}")
    print(f"  A - {example['A']}")
    print(f"  B - {example['B']}")
    print(f"  C - {example['C']}")
    print(f"  D - {example['D']}")
    print(f"  E - {example['E']}")
    print(f"\nGround truth: option {[example['answer']]}")


In [18]:
show_one(train_data.iloc[0])

Context: El trabajo es en primer término un proceso entre la naturaleza y el hombre,proceso en que este realiza,regula y controla mediante su propia acción su intercambio de materias con la naturaleza. En este proceso,el hombre se enfrenta como un poder natural con la materia de la naturaleza. Pone en acción las fuerzas naturales que forman su corporeidad,los brazos y las piernas,la cabeza y la mano,para de ese modo asimilarse,bajo una forma útil para su propia vida,las materias que la naturaleza le brinda. Y a la par que de ese modo actúa sobre la naturaleza exterior a él y la transforma,transforma su propia naturaleza,desarrollando las potencias que dormitan en él y sometiendo el juego de sus fuerzas a su propia disciplina. Aquí no vamos a ocuparnos de las primeras formas de trabajo,formas instintivas y de tipo animal. Aquí,partimos del supuesto del trabajo plasmado ya bajo una forma en la que pertenece exclusivamente al hombre. Una araña ejecuta operaciones que semejan a las manipul

### **Preprocessing the data**

In [19]:
#model_checkpoint = "bert-base-uncased"
#model_checkpoint =  'dccuchile/bert-base-spanish-wwm-uncased'
model_checkpoint = "MMG/bert-base-spanish-wwm-cased-finetuned-sqac-finetuned-squad"
batch_size = 16

In [20]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained(model_checkpoint, use_fast=True)

tokenizer_config.json:   0%|          | 0.00/565 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/242k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/480k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

In [21]:
tokenizer("Hello, this one sentence!", "And this sentence goes with it.")

{'input_ids': [4, 27616, 30933, 1017, 20578, 27297, 1823, 11830, 1127, 5, 6002, 20578, 1823, 11830, 3292, 1019, 1004, 7481, 4290, 1009, 5], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

In [22]:
train_data.head()

,text,question,A,B,C,D,E,answer,reason
0,El trabajo es en primer término un proceso ent...,"Medularmente,el autor intenta dilucidar",las diferencias entre lo instintivo y lo plani...,la naturaleza del trabajo exclusivamente humano.,el carácter pernicioso del trabajo en la actua...,la supremacía de la naturaleza frente a la hum...,las etapas que componen el proceso productivo.,B,El autor busca caracterizar el trabajo humano ...
1,El trabajo es en primer término un proceso ent...,"En el texto,el término ASIMILARSE se puede ree...",apropiarse.,amoldarse.,ajustarse.,compararse.,confrontarse.,A,Pone en acción las fuerzas naturales que forma...
2,El trabajo es en primer término un proceso ent...,Si quisiéramos que una labor se desarrollara m...,atemorizar a los operarios.,pagar menos salarios.,automatizar el proceso.,esclavizar a los obreros.,motivar al trabajador.,E,La atención deberá ser más reconcentrada cuant...
3,El trabajo es en primer término un proceso ent...,Se infiere que la acción humana aventaja a la ...,planificado.,instintivo.,inmediato.,detallista.,vigoroso.,A,Hay algo en que el peor maestro de obras avent...
4,El trabajo es en primer término un proceso ent...,Resulta incompatible con el texto aseverar que...,es un proceso donde interactúan hombre y natur...,"en el caso humano,se supedita siempre a una fi...",implica siempre esfuerzo y desgaste de diverso...,genera que una atención creciente y sostenida ...,"en el caso humano,puede darse a niveles incons...",E,El obrero siempre aporta una voluntad conscien...


In [23]:
answers = ["A", "B", "C", "D", "E"]
def preprocess_function(examples):
    global answers
    first_sentences = [[context] *  5 for context in examples["text"]]
    question_headers = examples["question"]

    second_sentences = [ [ f"{header} {examples[answer][i]}" for option, answer in enumerate(answers)] for i, header in enumerate(question_headers)]
    # flatten everything
    first_sentences = sum(first_sentences, [])
    second_sentences = sum(second_sentences, [])
    # Tokenize
    tokenized_examples = tokenizer(first_sentences, second_sentences, truncation=True)
    # Un-flatten
    return {k: [v[i:i+5] for i in range(0, len(v), 5)] for k, v in tokenized_examples.items()}

In [24]:
examples = train_data[:5]
features = preprocess_function(examples)
print(len(features["input_ids"]), len(features["input_ids"][0]), [len(x) for x in features["input_ids"][0]])

5 5 [481, 477, 481, 481, 478]


In [25]:
idx = 3
[tokenizer.decode(features["input_ids"][idx][i]) for i in range(5)]

['[CLS] El trabajo es en primer término un proceso entre la naturaleza y el hombre, proceso en que este realiza, regula y controla mediante su propia acción su intercambio de materias con la naturaleza. En este proceso, el hombre se enfrenta como un poder natural con la materia de la naturaleza. Pone en acción las fuerzas naturales que forman su corporeidad, los brazos y las piernas, la cabeza y la mano, para de ese modo asimilarse, bajo una forma útil para su propia vida, las materias que la naturaleza le brinda. Y a la par que de ese modo actúa sobre la naturaleza exterior a él y la transforma, transforma su propia naturaleza, desarrollando las potencias que dormitan en él y sometiendo el juego de sus fuerzas a su propia disciplina. Aquí no vamos a ocuparnos de las primeras formas de trabajo, formas instintivas y de tipo animal. Aquí, partimos del supuesto del trabajo plasmado ya bajo una forma en la que pertenece exclusivamente al hombre. Una araña ejecuta operaciones que semejan a 

In [26]:
show_one(train_data.loc[3])

Context: El trabajo es en primer término un proceso entre la naturaleza y el hombre,proceso en que este realiza,regula y controla mediante su propia acción su intercambio de materias con la naturaleza. En este proceso,el hombre se enfrenta como un poder natural con la materia de la naturaleza. Pone en acción las fuerzas naturales que forman su corporeidad,los brazos y las piernas,la cabeza y la mano,para de ese modo asimilarse,bajo una forma útil para su propia vida,las materias que la naturaleza le brinda. Y a la par que de ese modo actúa sobre la naturaleza exterior a él y la transforma,transforma su propia naturaleza,desarrollando las potencias que dormitan en él y sometiendo el juego de sus fuerzas a su propia disciplina. Aquí no vamos a ocuparnos de las primeras formas de trabajo,formas instintivas y de tipo animal. Aquí,partimos del supuesto del trabajo plasmado ya bajo una forma en la que pertenece exclusivamente al hombre. Una araña ejecuta operaciones que semejan a las manipul

In [27]:
from datasets import load_dataset, load_metric

In [28]:
from datasets import Dataset
train_dataset = Dataset.from_pandas(train_data)
test_dataset = Dataset.from_pandas(test_data)

In [29]:
train_dataset[0]

{'text': 'El trabajo es en primer término un proceso entre la naturaleza y el hombre,proceso en que este realiza,regula y controla mediante su propia acción su intercambio de materias con la naturaleza. En este proceso,el hombre se enfrenta como un poder natural con la materia de la naturaleza. Pone en acción las fuerzas naturales que forman su corporeidad,los brazos y las piernas,la cabeza y la mano,para de ese modo asimilarse,bajo una forma útil para su propia vida,las materias que la naturaleza le brinda. Y a la par que de ese modo actúa sobre la naturaleza exterior a él y la transforma,transforma su propia naturaleza,desarrollando las potencias que dormitan en él y sometiendo el juego de sus fuerzas a su propia disciplina. Aquí no vamos a ocuparnos de las primeras formas de trabajo,formas instintivas y de tipo animal. Aquí,partimos del supuesto del trabajo plasmado ya bajo una forma en la que pertenece exclusivamente al hombre. Una araña ejecuta operaciones que semejan a las manipu

In [30]:
test_dataset[0]

{'text': "No sabemos si Dios existe. Por esto se plantea la cuestión de si hemos de creer en Él o no. ¿Por qué hay algo y no más bien nada? No lo sabemos. Nunca lo sabremos. ¿Para qué (con qué fin)? Tampoco lo sabemos,ni siquiera sabemos si hay un fin. Pero si bien es verdad que de la nada no nace nada,la mera existencia de algo ―el mundo,el universo― parece implicar que siempre ha habido algo': que el ser es eterno,increado,tal vez creador,y es a esto a lo que algunos llaman Dios. ¿Existe Dios? Existe por definición,así podemos tomar su definición por una demostración. Esto es lo que hay de fascinante y de irritante a la vez en la famosa prueba ontológica,que atraviesa el conjunto de la filosofía occidental. ¿Cómo se define a Dios? Como el ser supremo (san Anselmo': «el ser en relación con el cual es imposible concebir nada más grande»),el ser soberanamente perfecto (Descartes),el ser absolutamente infinito (Spinoza,Hegel). Ahora bien,si no existiera,no sería ni el más grande ni realm

In [31]:
tokenized_train_dataset = train_dataset.map(preprocess_function, batched=True)
tokenized_test_dataset = test_dataset.map(preprocess_function, batched=True)

Map:   0%|          | 0/6279 [00:00<?, ? examples/s]

Map:   0%|          | 0/724 [00:00<?, ? examples/s]

## **Fine-tuning the model**

In [32]:
from transformers import AutoModelForMultipleChoice, TrainingArguments, Trainer

model = AutoModelForMultipleChoice.from_pretrained(model_checkpoint)

config.json:   0%|          | 0.00/725 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/437M [00:00<?, ?B/s]

Some weights of BertForMultipleChoice were not initialized from the model checkpoint at MMG/bert-base-spanish-wwm-cased-finetuned-sqac-finetuned-squad and are newly initialized: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [33]:
from dataclasses import dataclass
from transformers.tokenization_utils_base import PreTrainedTokenizerBase, PaddingStrategy
from typing import Optional, Union
import torch

In [34]:
tokenized_train_dataset

Dataset({
    features: ['text', 'question', 'A', 'B', 'C', 'D', 'E', 'answer', 'reason', 'input_ids', 'token_type_ids', 'attention_mask'],
    num_rows: 6279
})

In [35]:
tokenized_test_dataset

Dataset({
    features: ['text', 'question', 'A', 'B', 'C', 'D', 'E', 'answer', 'reason', 'input_ids', 'token_type_ids', 'attention_mask'],
    num_rows: 724
})

In [36]:
@dataclass
class DataCollatorForMultipleChoice:
    """
    Data collator that will dynamically pad the inputs for multiple choice received.
    """

    tokenizer: PreTrainedTokenizerBase
    padding: Union[bool, str, PaddingStrategy] = True
    max_length: Optional[int] = None
    pad_to_multiple_of: Optional[int] = None

    def __call__(self, features):
        label_name = "label" if "label" in features[0].keys() else "labels"
        labels = [feature.pop(label_name) for feature in features]
        label_map = {"A": 0, "B": 1, "C": 2, "D": 3, "E": 4}
        labels = [label_map[label] for label in labels]



        batch_size = len(features)
        num_choices = len(features[0]["input_ids"])
        flattened_features = [[{k: v[i] for k, v in feature.items()} for i in range(num_choices)] for feature in features]
        flattened_features = sum(flattened_features, [])

        batch = self.tokenizer.pad(
            flattened_features,
            padding=self.padding,
            max_length=self.max_length,
            pad_to_multiple_of=self.pad_to_multiple_of,
            return_tensors="pt",
        )

        # Un-flatten
        batch = {k: v.view(batch_size, num_choices, -1) for k, v in batch.items()}
        # Add back labels
        batch["labels"] = torch.tensor(labels, dtype=torch.int64)
        return batch

In [37]:
def rename_answer_to_label(example):
    example["label"] = example.pop("answer")
    return example
tokenized_train_dataset = tokenized_train_dataset.map(rename_answer_to_label)


Map:   0%|          | 0/6279 [00:00<?, ? examples/s]

In [38]:
tokenized_test_dataset = tokenized_test_dataset.map(rename_answer_to_label)

Map:   0%|          | 0/724 [00:00<?, ? examples/s]

In [39]:
accepted_keys = ["input_ids", "attention_mask", "label"]
features = [{k: v for k, v in tokenized_train_dataset[i].items() if k in accepted_keys} for i in range(10)]
batch = DataCollatorForMultipleChoice(tokenizer)(features)

You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


In [40]:
[tokenizer.decode(batch["input_ids"][8][i].tolist()) for i in range(4)]

['[CLS] Aunque se escandalicen los adoradores de mitos i de fraseolojías tradicionales, conviene prescindir de cuestiones sobre fundamentos del Estado i principios del Gobierno i repetir con un verdadero pensador [UNK] : [UNK] cualquier Gobierno, con la mayor suma de garantías individuales i lo menos posible de acción administrativa. Al comparar las garantías que el súbdito inglés disfruta en la Gran Bretaña con las vejaciones que el ciudadano sufre en el Perú, se comprende que las formas de Gobierno nada o mui poco significan para la libertad del individuo. ¿ Qué vale más [UNK] : [UNK] habitar en una autocracia rejida por un Marco Aurelio o en una república gobernada por un Cáceres o un Piérola? Hai que mostrar al pueblo el horror de su envilecimiento i de su miseria ; nunca se verificó escelente autopsia sin despedazar el cadáver, ni se conoció a fondo una sociedad sin descarnar su esqueleto. ¿ Por qué asustarse o escandalizarse? Cuanto se diga ¿ no lo palpan nacionales i estranjeros

In [41]:
show_one(train_dataset[8])

Context: Aunque se escandalicen los adoradores de mitos i de fraseolojías tradicionales,conviene prescindir de cuestiones sobre fundamentos del Estado i principios del Gobierno i repetir con un verdadero pensador':' cualquier Gobierno,con la mayor suma de garantías individuales i lo menos posible de acción administrativa. Al comparar las garantías que el súbdito inglés disfruta en la Gran Bretaña con las vejaciones que el ciudadano sufre en el Perú,se comprende que las formas de Gobierno nada o mui poco significan para la libertad del individuo. ¿Qué vale más':' habitar en una autocracia rejida por un Marco Aurelio o en una república gobernada por un Cáceres o un Piérola? Hai que mostrar al pueblo el horror de su envilecimiento i de su miseria; nunca se verificó escelente autopsia sin despedazar el cadáver,ni se conoció a fondo una sociedad sin descarnar su esqueleto. ¿Por qué asustarse o escandalizarse? Cuanto se diga ¿no lo palpan nacionales i estranjeros? La lepra no se cura escondi

## **Define metric**

In [42]:
import numpy as np

def compute_metrics(eval_predictions):
    predictions, label_ids = eval_predictions
    preds = np.argmax(predictions, axis=1)
    return {"accuracy": (preds == label_ids).astype(np.float32).mean().item()}

In [43]:
model_name = model_checkpoint.split("/")[-1]

args = TrainingArguments(
    f"{model_name}-exams-peru",
    #output_dir=f"/content/mymodel",
    evaluation_strategy = "epoch",
    #learning_rate=5e-5,
    learning_rate=5e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=4,
    weight_decay=0.01,
    push_to_hub=False,
    gradient_checkpointing=True,
    save_steps=500,
    save_total_limit=3,
    fp16=True,
)

In [44]:
trainer = Trainer(
    model,
    args,
    train_dataset=tokenized_train_dataset,
    eval_dataset=tokenized_test_dataset,
    tokenizer=tokenizer,
    data_collator=DataCollatorForMultipleChoice(tokenizer),
    compute_metrics=compute_metrics,

)

In [45]:
trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy
1,No log,1.239325,0.520718
2,1.309800,1.153406,0.570442
3,0.663000,1.546244,0.610497
4,0.231200,2.042591,0.609116


TrainOutput(global_step=1572, training_loss=0.7071013753954084, metrics={'train_runtime': 4063.6213, 'train_samples_per_second': 6.181, 'train_steps_per_second': 0.387, 'total_flos': 3.304118966593536e+16, 'train_loss': 0.7071013753954084, 'epoch': 4.0})

In [46]:
trainer.save_model(f"./{model_name}-exams-peru")

In [47]:
tokenizer.save_pretrained(f"./{model_name}-exams-peru")

('./bert-base-spanish-wwm-cased-finetuned-sqac-finetuned-squad-exams-peru/tokenizer_config.json',
 './bert-base-spanish-wwm-cased-finetuned-sqac-finetuned-squad-exams-peru/special_tokens_map.json',
 './bert-base-spanish-wwm-cased-finetuned-sqac-finetuned-squad-exams-peru/vocab.txt',
 './bert-base-spanish-wwm-cased-finetuned-sqac-finetuned-squad-exams-peru/added_tokens.json',
 './bert-base-spanish-wwm-cased-finetuned-sqac-finetuned-squad-exams-peru/tokenizer.json')

### Test

In [48]:
from transformers import AutoModelForMultipleChoice, AutoTokenizer

# Cargar el modelo y el tokenizador
model_load = AutoModelForMultipleChoice.from_pretrained(f"./{model_name}-exams-peru")
tokenizer_load  = AutoTokenizer.from_pretrained(f"./{model_name}-exams-peru")

In [49]:
model_load.eval()

BertForMultipleChoice(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(31002, 768, padding_idx=1)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, ele

In [50]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [51]:
summit_data.head()

,text,question,A,B,C,D,E,answer,reason
0,La identificación de autores y el estudio de l...,La finalidad del autor del texto es,señalar la importancia de la visión universali...,proponer que las literaturas regionales están ...,denunciar la visión centralista y hegemónica d...,señalar el gran auge del fenómeno regionalista...,aclarar el sentido y la importancia de los est...,E,La intención del autor es aclarar el sentido e...
1,La identificación de autores y el estudio de l...,"En el texto,el verbo ENTRONIZAR alude a",una posición privilegiada.,una muestra de solidez.,la adquisición de fortuna.,un signo de inclusión.,una fama inmerecida.,A,Se refiere a que estos discursos ocupan una po...
2,La identificación de autores y el estudio de l...,"En el texto,el término CUERPO alude a",sensualidad.,títulos.,novelas.,extrañeza.,conjunto.,E,"Se refiere al conjunto de obras literarias.,,,..."
3,La identificación de autores y el estudio de l...,Marque la alternativa que es incompatible con ...,Hay quienes consideran que las literaturas reg...,La literatura regional pretende fortalecer los...,La constitución del canon literario halla su f...,Las visiones periféricas del mundo no suelen s...,Mostrar las expresiones autóctonas fue uno de ...,B,Es el regionalismo el que tiene ese objetivo. ...
4,La identificación de autores y el estudio de l...,Si “literatura regional” fuese sinónimo de “re...,resaltarían las características vernáculas de ...,destacarían la influencia de los escritores ex...,destacarían la influencia de los escritores na...,criticarían las propuestas que fuesen ajenas a...,adoptarían una perspectiva latinoamericanista ...,A,"Si fuesen sinónimos,entonces la “literatura re..."


In [52]:
summit_dataset = Dataset.from_pandas(summit_data)

In [53]:
tokenized_summit_dataset = summit_dataset.map(preprocess_function, batched=True)

Map:   0%|          | 0/720 [00:00<?, ? examples/s]

In [54]:
tokenized_summit_dataset = tokenized_summit_dataset.map(rename_answer_to_label)

Map:   0%|          | 0/720 [00:00<?, ? examples/s]

In [55]:
print(summit_data.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 720 entries, 0 to 719
Data columns (total 9 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   text      720 non-null    object
 1   question  720 non-null    object
 2   A         720 non-null    object
 3   B         720 non-null    object
 4   C         720 non-null    object
 5   D         720 non-null    object
 6   E         720 non-null    object
 7   answer    720 non-null    object
 8   reason    720 non-null    object
dtypes: object(9)
memory usage: 50.8+ KB
None


In [56]:
tokenized_summit_dataset

Dataset({
    features: ['text', 'question', 'A', 'B', 'C', 'D', 'E', 'reason', 'input_ids', 'token_type_ids', 'attention_mask', 'label'],
    num_rows: 720
})

In [57]:
predictions = trainer.predict(tokenized_summit_dataset)

In [58]:
logits, label_ids = predictions.predictions, predictions.label_ids

preds = np.argmax(logits, axis=1)

In [59]:
accuracy = compute_metrics((logits, label_ids))["accuracy"]
print(f"Precisión: {accuracy}")

Precisión: 0.5666666626930237


In [60]:
import numpy as np

index_to_label = {0: "A", 1: "B", 2: "C", 3: "D", 4: "E"}
predicted_answers = [index_to_label[pred] for pred in preds]

with open("predictions.txt", "w") as file:
    for answer in predicted_answers:
        file.write(f"{answer}\n")
print("Predicciones guardadas en 'predictions.txt'.")

Predicciones guardadas en 'predictions.txt'.
